# Prep

In [1]:
print("Playground notebook")

Playground notebook


In [2]:
%notebook inline

In [3]:
from pymatgen.electronic_structure.boltztrap2 import *
from monty.serialization import loadfn
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.electronic_structure.plotter import DosPlotter

import matplotlib
orig_mpl_backend = plt.get_backend()
orig_mpl_backend = 'agg'
matplotlib.use('pgf')
plt.rcParams.update({
    "font.family": "serif",  # use serif/main font for text elements
    "text.usetex": True,     # use inline math for ticks
    "pgf.rcfonts": False,    # don't setup fonts from rc parameters
    "pgf.preamble": "\n".join([
         r"\usepackage{unicode-math}",   # unicode math setup
    ])
})

In [4]:
def fp_for_element(el):
    el = el.replace(' ', '_')
    if '-' in el:
        a, b = el.split('-')
        return f'{a}-defect-data/{b}/'
    
    return f'{el}/'

In [75]:
elements = {
    'Mo3Si2': ['MoSi', 'SiMo', 'VMo', 'VSi'],
    'MoSi2 tetragonal': [],
    'MoSi2 hexagonal': [],
}

In [113]:
# deviation from Fermi energy in eV
# energy_range = 3.5
energy_range = 10
erange = [-energy_range, energy_range/2]
eranget = (-energy_range, energy_range/2)
temp_r = np.arange(300,1300,300)
temps = [300, 600, 900, 1200]
temp = [temps[-1]]
dopings_r = 10.**np.arange(18,23,0.15)
dopings_arr = [dopings_r[10], dopings_r[20], dopings_r[30]]
doping_type = 'p'

save_to_disk = False
load_from_disk = not save_to_disk

# MyBztPlotter

In [64]:
class MyBztPlotter(BztPlotter):
    def plot_props_mul(
        self,
        prop_y,
        prop_x,
        prop_z="temp",
        output="avg_eigs",
        dop_type="n",
        doping=None,
        temps=None,
        xlim=(-2, 2),
        ax=None,
        label="",
        linestyle="solid",
        marker="o",
        markevery=1,
        color="b",
    ):
        """
        Function to plot the transport properties.
        Args:
            prop_y: property to plot among ("Conductivity","Seebeck","Kappa","Carrier_conc",
                "Hall_carrier_conc_trace"). Abbreviations are possible, like "S" for "Seebeck"
            prop_x: independent variable in the x-axis among ('mu','doping','temp')
            prop_z: third variable to plot multiple curves ('doping','temp')
            output: 'avg_eigs' to plot the average of the eigenvalues of the properties
                tensors; 'eigs' to plot the three eigenvalues of the properties
                tensors.
            dop_type: 'n' or 'p' to specify the doping type in plots that use doping
                levels as prop_x or prop_z
            doping: list of doping level to plot, useful to reduce the number of curves
                when prop_z='doping'
            temps: list of temperatures to plot, useful to reduce the number of curves
                when prop_z='temp'
            xlim: chemical potential range in eV, useful when prop_x='mu'
            ax: figure.axes where to plot. If None, a new figure is produced.
        Example:
        bztPlotter.plot_props('S','mu','temp',temps=[600,900,1200]).show()
        more example are provided in the notebook
        "How to use Boltztra2 interface.ipynb".
        """
        props = (
            "Conductivity",
            "Seebeck",
            "Kappa",
            "Effective_mass",
            "Power_Factor",
            "Carrier_conc",
            "Hall_carrier_conc_trace",
        )
        props_lbl = (
            "Conductivity",
            "Seebeck",
            "$K_{el}$",
            "Effective mass",
            "Power Factor",
            "Carrier concentration",
            "Hall carrier conc.",
        )
        props_unit = (
            r"$(\mathrm{S\,m^{-1}})$",
            r"($\mu$V/K)",
            r"$(W / (m \cdot K))$",
            r"$(m_e)$",
            r"$( mW / (m\cdot K^2)$",
            r"$(cm^{-3})$",
            r"$(cm^{-3})$",
        )

        props_short = [p[: len(prop_y)] for p in props]

        if prop_y not in props_short:
            raise BoltztrapError("prop_y not valid")

        if prop_x not in ("mu", "doping", "temp"):
            raise BoltztrapError("prop_x not valid")

        if prop_z not in ("doping", "temp"):
            raise BoltztrapError("prop_z not valid")

        idx_prop = props_short.index(prop_y)

        leg_title = ""

        mu = self.bzt_transP.mu_r_eV

        if prop_z == "doping" and prop_x == "temp":
            p_array = eval("self.bzt_transP." + props[idx_prop] + "_" + prop_z)
        else:
            p_array = eval("self.bzt_transP." + props[idx_prop] + "_" + prop_x)

        if ax is None:
            plt.figure(figsize=(10, 8))

        temps_all = self.bzt_transP.temp_r.tolist()
        if temps is None:
            temps = self.bzt_transP.temp_r.tolist()

        if isinstance(self.bzt_transP.doping, np.ndarray):
            doping_all = self.bzt_transP.doping.tolist()
            if doping is None:
                doping = doping_all

        # special case of carrier and hall carrier concentration 2d arrays (temp,mu)
        if idx_prop in [5, 6]:
            if prop_z == "temp" and prop_x == "mu":
                for temp in temps:
                    ti = temps_all.index(temp)
                    prop_out = p_array[ti] if idx_prop == 6 else np.abs(p_array[ti])
                    plt.semilogy(mu, prop_out, label=label + ": " + str(temp) + " K", linestyle=linestyle, marker=marker, markevery=markevery, color=color)

                plt.xlabel(r"$\mu$ (eV)", fontsize=30)
                plt.xlim(xlim)
            else:
                raise BoltztrapError(
                    "only prop_x=mu and prop_z=temp are \
                    available for c.c. and Hall c.c.!"
                )

        elif prop_z == "temp" and prop_x == "mu":
            for temp in temps:
                ti = temps_all.index(temp)
                prop_out = np.linalg.eigh(p_array[ti])[0]
                if output == "avg_eigs":
                    plt.plot(mu, prop_out.mean(axis=1), label=label + ": " + str(temp) + " K", linestyle=linestyle, marker=marker, markevery=markevery, color=color)
                elif output == "eigs":
                    for i in range(3):
                        plt.plot(
                            mu,
                            prop_out[:, i],
                            label=label + ": " + "eig " + str(i) + " " + str(temp) + " K", linestyle=linestyle, marker=marker, markevery=markevery, color=color
                        )

            plt.xlabel(r"$\mu$ (eV)", fontsize=30)
            plt.xlim(xlim)

        elif prop_z == "temp" and prop_x == "doping":
            for temp in temps:
                ti = temps_all.index(temp)
                prop_out = np.linalg.eigh(p_array[dop_type][ti])[0]
                if output == "avg_eigs":
                    plt.semilogx(doping_all, prop_out.mean(axis=1), label=label + ": " + str(temp) + " K", linestyle=linestyle, marker=marker, markevery=markevery, color=color)
                elif output == "eigs":
                    for i in range(3):
                        plt.plot(
                            doping_all,
                            prop_out[:, i],
                            label=label + ": " + "eig " + str(i) + " " + str(temp) + " K", linestyle=linestyle, marker=marker, markevery=markevery, color=color
                        )
            plt.xlabel(r"Carrier conc. $cm^{-3}$", fontsize=30)
            leg_title = dop_type + "-type"

        elif prop_z == "doping" and prop_x == "temp":
            for dop in doping:
                di = doping_all.index(dop)
                prop_out = np.linalg.eigh(p_array[dop_type][:, di])[0]
                if output == "avg_eigs":
                    plt.plot(
                        temps_all,
                        prop_out.mean(axis=1),
                        label=label + ": " + np.format_float_scientific(dop, unique=False, precision=2) + " $cm^{-3}$", linestyle=linestyle, marker=marker, markevery=markevery, color=color
                    )
                elif output == "eigs":
                    for i in range(3):
                        plt.plot(
                            temps_all,
                            prop_out[:, i],
                            label=label + ": " + "eig " + str(i) + " " + np.format_float_scientific(dop, unique=False, precision=2) + " $cm^{-3}$", linestyle=linestyle, marker=marker, markevery=markevery, color=color
                        )

            plt.xlabel(r"Temperature (K)", fontsize=30)
            leg_title = dop_type + "-type"

        plt.ylabel(props_lbl[idx_prop] + " " + props_unit[idx_prop], fontsize=30)
        plt.xticks(fontsize=25)
        plt.yticks(fontsize=25)
        plt.legend(title=leg_title, fontsize=15)
        plt.tight_layout()
        plt.grid()
        return plt
    
    def plot_props(
        self,
        prop_y,
        prop_x,
        prop_z="temp",
        output="avg_eigs",
        dop_type="n",
        doping=None,
        temps=None,
        xlim=(-2, 2),
        ax=None,
        markevery=1,
    ):
        """
        Function to plot the transport properties.
        Args:
            prop_y: property to plot among ("Conductivity","Seebeck","Kappa","Carrier_conc",
                "Hall_carrier_conc_trace"). Abbreviations are possible, like "S" for "Seebeck"
            prop_x: independent variable in the x-axis among ('mu','doping','temp')
            prop_z: third variable to plot multiple curves ('doping','temp')
            output: 'avg_eigs' to plot the average of the eigenvalues of the properties
                tensors; 'eigs' to plot the three eigenvalues of the properties
                tensors.
            dop_type: 'n' or 'p' to specify the doping type in plots that use doping
                levels as prop_x or prop_z
            doping: list of doping level to plot, useful to reduce the number of curves
                when prop_z='doping'
            temps: list of temperatures to plot, useful to reduce the number of curves
                when prop_z='temp'
            xlim: chemical potential range in eV, useful when prop_x='mu'
            ax: figure.axes where to plot. If None, a new figure is produced.
        Example:
        bztPlotter.plot_props('S','mu','temp',temps=[600,900,1200]).show()
        more example are provided in the notebook
        "How to use Boltztra2 interface.ipynb".
        """
        props = (
            "Conductivity",
            "Seebeck",
            "Kappa",
            "Effective_mass",
            "Power_Factor",
            "Carrier_conc",
            "Hall_carrier_conc_trace",
        )
        props_lbl = (
            "Conductivity",
            "Seebeck",
            "$K_{el}$",
            "Effective mass",
            "Power Factor",
            "Carrier concentration",
            "Hall carrier conc.",
        )
        props_unit = (
            r"$(\mathrm{S\,m^{-1}})$",
            r"($\mu$V/K)",
            r"$(W / (m \cdot K))$",
            r"$(m_e)$",
            r"$( mW / (m\cdot K^2)$",
            r"$(cm^{-3})$",
            r"$(cm^{-3})$",
        )

        props_short = [p[: len(prop_y)] for p in props]

        if prop_y not in props_short:
            raise BoltztrapError("prop_y not valid")

        if prop_x not in ("mu", "doping", "temp"):
            raise BoltztrapError("prop_x not valid")

        if prop_z not in ("doping", "temp"):
            raise BoltztrapError("prop_z not valid")
            
        colors = ['#377eb8', '#ff7f00', '#4daf4a',
           '#f781bf', '#a65628', '#984ea3',
           '#999999', '#e41a1c', '#dede00']

        linestyles = [
            ('solid', 'solid'),      # Same as (0, ()) or '-'
             ('dotted', 'dotted'),    # Same as (0, (1, 1)) or ':'
             ('dashed', 'dashed'),    # Same as '--'
             ('dashdot', 'dashdot'),  # Same as '-.'
             ('long dash with offset', (5, (10, 3))),
             ('densely dashed',        (0, (5, 1))),

             ('densely dashdotted',    (0, (3, 1, 1, 1))),

             ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
             ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
             ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))]

        markers = ['o', 'v', 's',
                  'X', 'd', '*',
                  'x', 'P', '2']

        idx_prop = props_short.index(prop_y)

        leg_title = ""

        mu = self.bzt_transP.mu_r_eV

        if prop_z == "doping" and prop_x == "temp":
            p_array = eval("self.bzt_transP." + props[idx_prop] + "_" + prop_z)
        else:
            p_array = eval("self.bzt_transP." + props[idx_prop] + "_" + prop_x)

        if ax is None:
            plt.figure(figsize=(10, 8))

        temps_all = self.bzt_transP.temp_r.tolist()
        if temps is None:
            temps = self.bzt_transP.temp_r.tolist()

        if isinstance(self.bzt_transP.doping, np.ndarray):
            doping_all = self.bzt_transP.doping.tolist()
            if doping is None:
                doping = doping_all

        # special case of carrier and hall carrier concentration 2d arrays (temp,mu)
        if idx_prop in [5, 6]:
            if prop_z == "temp" and prop_x == "mu":
                for i, temp in enumerate(temps):
                    ti = temps_all.index(temp)
                    prop_out = p_array[ti] if idx_prop == 6 else np.abs(p_array[ti])
                    plt.semilogy(mu, prop_out, label=str(temp) + " K", linestyle=linestyles[i][1], marker=markers[i], markevery=markevery, color=colors[i])

                plt.xlabel(r"$\mu$ (eV)", fontsize=30)
                plt.xlim(xlim)
            else:
                raise BoltztrapError(
                    "only prop_x=mu and prop_z=temp are \
                    available for c.c. and Hall c.c.!"
                )

        elif prop_z == "temp" and prop_x == "mu":
            for i, temp in enumerate(temps):
                ti = temps_all.index(temp)
                prop_out = np.linalg.eigh(p_array[ti])[0]
                if output == "avg_eigs":
                    plt.plot(mu, prop_out.mean(axis=1), label=str(temp) + " K", linestyle=linestyles[i][1], marker=markers[i], markevery=markevery, color=colors[i])
                elif output == "eigs":
                    for i in range(3):
                        plt.plot(
                            mu,
                            prop_out[:, i],
                            label="eig " + str(i) + " " + str(temp) + " K", linestyle=linestyles[i][1], marker=markers[i], markevery=markevery, color=colors[i]
                        )

            plt.xlabel(r"$\mu$ (eV)", fontsize=30)
            plt.xlim(xlim)

        elif prop_z == "temp" and prop_x == "doping":
            for j, temp in enumerate(temps):
                ti = temps_all.index(temp)
                prop_out = np.linalg.eigh(p_array[dop_type][ti])[0]
                if output == "avg_eigs":
                    plt.semilogx(doping_all, prop_out.mean(axis=1), label=str(temp) + " K", linestyle=linestyles[j][1], marker=markers[j], markevery=markevery, color=colors[j])
                elif output == "eigs":
                    for i in range(3):
                        plt.plot(
                            doping_all,
                            prop_out[:, i],
                            label="eig " + str(i) + " " + str(temp) + " K", linestyle=linestyles[i][1], marker=markers[i], markevery=markevery, color=colors[i]
                        )
            plt.xlabel(r"Carrier conc. $cm^{-3}$", fontsize=30)
            leg_title = dop_type + "-type"

        elif prop_z == "doping" and prop_x == "temp":
            for j, dop in enumerate(doping):
                di = doping_all.index(dop)
                prop_out = np.linalg.eigh(p_array[dop_type][:, di])[0]
                if output == "avg_eigs":
                    plt.plot(
                        temps_all,
                        prop_out.mean(axis=1),
                        label=np.format_float_scientific(dop, unique=False, precision=2) + " $cm^{-3}$", linestyle=linestyles[j][1], marker=markers[j], markevery=markevery, color=colors[j]
                    )
                elif output == "eigs":
                    for i in range(3):
                        plt.plot(
                            temps_all,
                            prop_out[:, i],
                            label="eig " + str(i) + " " + np.format_float_scientific(dop, unique=False, precision=2) + " $cm^{-3}$", linestyle=linestyles[i][1], marker=markers[i], markevery=markevery, color=colors[i]
                        )

            plt.xlabel(r"Temperature (K)", fontsize=30)
            leg_title = dop_type + "-type"

        plt.ylabel(props_lbl[idx_prop] + " " + props_unit[idx_prop], fontsize=30)
        plt.xticks(fontsize=25)
        plt.yticks(fontsize=25)
        plt.legend(title=leg_title, fontsize=15)
        plt.tight_layout()
        plt.grid()
        return plt

# Functions

In [76]:
def gather_element_data(el):
    vrun = Vasprun(f'{fp_for_element(el)}vasprun.xml', parse_projected_eigen=True)
    data = VasprunBSLoader(vrun)
    
    if save_to_disk:
        bztInterp = BztInterpolator(data,lpfac=10,energy_range=energy_range,curvature=True,save_bztInterp=True, fname=f'{fp_for_element(el)}bztInterp.json.gz')
    elif load_from_disk:
        bztInterp = BztInterpolator(data, load_bztInterp=True, fname=f'{fp_for_element(el)}bztInterp.json.gz')
        
    if save_to_disk:
        bztTransp = BztTransportProperties(bztInterp, temp_r=temp_r, doping=dopings_r, save_bztTranspProps=True, fname=f'{fp_for_element(el)}bztTranspProps.json.gz')
    elif load_from_disk:
        bztTransp = BztTransportProperties(bztInterp, load_bztTranspProps=True, fname=f'{fp_for_element(el)}bztTranspProps.json.gz')
        
    bztTransp.compute_properties_doping(doping=dopings_r)
    
    bztPlotter = MyBztPlotter(bztTransp,bztInterp)
    
    return {
        'data': data,
        'interp': bztInterp,
        'transp': bztTransp,
        'plotter': bztPlotter
    }

element_data = {}
for k, v in elements.items():
    for defect_name in v:
        el_data = gather_element_data(f'{k}-{defect_name}')
        element_data[f'{k}-{defect_name}'] = el_data
    
    el_data = gather_element_data(k)
    element_data[k] = el_data

/Users/max/opt/anaconda3/envs/boltztrap2/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1732: UserWarning: Ignoring unknown variable type SHA256
  warnings.warn(f"Ignoring unknown variable type {key}")
/Users/max/opt/anaconda3/envs/boltztrap2/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1732: UserWarning: Ignoring unknown variable type COPYR
  warnings.warn(f"Ignoring unknown variable type {key}")


In [19]:
def make_plt():
    return plt.figure(figsize=(10, 8))

def save_plt(ax, name):
    matplotlib.use('pgf')
    ax.savefig(f'figures/{name}.pdf', backend='pgf')
    ax.close()

# Plots

In [20]:
colors = ['#377eb8', '#ff7f00', '#4daf4a',
           '#f781bf', '#a65628', '#984ea3',
           '#999999', '#e41a1c', '#dede00']

linestyles = [
    ('solid', 'solid'),      # Same as (0, ()) or '-'
     ('dotted', 'dotted'),    # Same as (0, (1, 1)) or ':'
     ('dashed', 'dashed'),    # Same as '--'
     ('dashdot', 'dashdot'),  # Same as '-.'
     ('long dash with offset', (5, (10, 3))),
     ('densely dashed',        (0, (5, 1))),

     ('densely dashdotted',    (0, (3, 1, 1, 1))),

     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))]

markers = ['o', 'v', 's',
          'X', 'd', '*',
          'x', 'P', '2']

## Partial DoS

In [105]:
tot_proj_dosMo3Si2 = element_data['Mo3Si2']['interp'].get_dos(partial_dos=True, progress=True)

200it [03:07,  1.07it/s]                                                        


In [202]:
pltdos = DosPlotter(sigma=0.03)
pltdos.add_dos_dict(tot_proj_dosMo3Si2.get_element_dos())
p = pltdos.get_plot()
p.xlim(eranget)
p.gcf().set_size_inches(12, 3)
p.ylabel('DoS')
p.locator_params(axis='y', nbins=6)
save_plt(p, "partial_dos_Mo3Si2")

In [106]:
tot_proj_dosMoSi2T = element_data['MoSi2 tetragonal']['interp'].get_dos(partial_dos=True, progress=True)
pltdos = DosPlotter(sigma=0.004, fig_size=(10,4))
pltdos.add_dos_dict(tot_proj_dosMoSi2T.get_element_dos())
save_plt(pltdos.get_plot(), "partial_dos_MoSi2_tetragonal")

60it [16:17, 16.29s/it]                                                         


In [107]:
tot_proj_dosMoSi2H = element_data['MoSi2 hexagonal']['interp'].get_dos(partial_dos=True, progress=True)
pltdos = DosPlotter(sigma=0.004)
pltdos.add_dos_dict(tot_proj_dosMoSi2H.get_element_dos())
save_plt(pltdos.get_plot(), "partial_dos_MoSi2_hexagonal")

180it [02:52,  1.04it/s]                                                        


## Band structure

In [102]:
kpaths = [["Γ", "X", "M", "Γ", "Z", "R", "A", "Z"], ["X", "R"], ["M", "A"]]

kp_lbl = {"A":np.array([0.5000000000, 0.5000000000, 0.5000000000]),
          "Γ":np.array([0.0000000000, 0.0000000000, 0.0000000000]),
          "M":np.array([0.5000000000, 0.5000000000, 0.0000000000]),
          "R":np.array([0.0000000000, 0.5000000000, 0.5000000000]),
          "X":np.array([0.0000000000, 0.5000000000, 0.0000000000]),
          "Z":np.array([0.0000000000, 0.0000000000, 0.5000000000]),
          }

sbs = element_data['Mo3Si2']['interp'].get_band_structure(kpaths,kp_lbl)
BSPlotter(sbs).show(ylim=[-energy_range, energy_range/2])
save_plt(BSPlotter(sbs).get_plot(ylim=[-energy_range, energy_range/2]), "bs_Mo3Si2")

/Users/max/opt/anaconda3/envs/boltztrap2/lib/python3.9/site-packages/pymatgen/electronic_structure/plotter.py:745: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [103]:
kpaths = [["Γ", "X", "P", "N", "Γ", "M", "S"], ["S0", "Γ"], ["X", "R"], ["G", "M"]]

kp_lbl = {"G":np.array([0.5000000000, 0.5000000000, -0.0834984314]),
          "Γ":np.array([0.0000000000, 0.0000000000, 0.0000000000]),
          "M":np.array([0.5000000000, 0.5000000000, -0.5000000000]),
          "N":np.array([0.0000000000, 0.5000000000, 0.0000000000]),
          "P":np.array([0.2500000000, 0.2500000000, 0.2500000000]),
          "R":np.array([-0.0834984314, 0.0834984314, 0.5000000000]),
          "S":np.array([0.2917492157, 0.7082507843, -0.2917492157]),
          "S0":np.array([-0.2917492157, 0.2917492157, 0.2917492157]),
          "X":np.array([0.0000000000, 0.0000000000, 0.5000000000]),
          }

sbs = element_data['MoSi2 tetragonal']['interp'].get_band_structure(kpaths,kp_lbl)
BSPlotter(sbs).show(ylim=[-energy_range, energy_range/2])
save_plt(BSPlotter(sbs).get_plot(ylim=[-energy_range, energy_range/2]), "bs_MoSi2_tetragonal")

/Users/max/opt/anaconda3/envs/boltztrap2/lib/python3.9/site-packages/pymatgen/electronic_structure/plotter.py:745: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [104]:
kpaths = [["Γ", "M", "K", "Γ", "A", "L", "H", "A"], ["L", "M"], ["H", "K"]]

kp_lbl = {"A":np.array([0.0000000000, 0.0000000000, 0.5000000000]),
          "A'":np.array([-0.0000000000, -0.0000000000, -0.5000000000]),
          "Γ":np.array([0.0000000000, 0.0000000000, 0.0000000000]),
          "H":np.array([0.3333333333, 0.3333333333, 0.5000000000]),
          "H'":np.array([-0.3333333333, -0.3333333333, -0.5000000000]),
          "H2":np.array([0.3333333333, 0.3333333333, -0.5000000000]),
          "H2'":np.array([-0.3333333333, -0.3333333333, 0.5000000000]),
          "K":np.array([0.3333333333, 0.3333333333, 0.0000000000]),
          "K'":np.array([-0.3333333333, -0.3333333333, -0.0000000000]),
          "L":np.array([0.5000000000, 0.0000000000, 0.5000000000]),
          "L'":np.array([-0.5000000000, -0.0000000000, -0.5000000000]),
          "M":np.array([0.5000000000, 0.0000000000, 0.0000000000]),
          "M'":np.array([-0.5000000000, -0.0000000000, -0.0000000000]),
          }

sbs = element_data['MoSi2 hexagonal']['interp'].get_band_structure(kpaths,kp_lbl)
BSPlotter(sbs).show(ylim=[-energy_range, energy_range/2])
save_plt(BSPlotter(sbs).get_plot(ylim=[-energy_range, energy_range/2]), "bs_MoSi2_hexagonal")

/Users/max/opt/anaconda3/envs/boltztrap2/lib/python3.9/site-packages/pymatgen/electronic_structure/plotter.py:745: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Energy comparisons

In [238]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('S','mu','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=100,color=colors[i],linestyle=linestyles[i][1])

ax.xlim(eranget)
ax.ylim((-300,250))

p.legend().remove()

p.gcf().set_size_inches(12, 3)
plt.tick_params(axis='x', left=False, top=False, right=False, bottom=False, labelleft=False, labeltop=False, labelright=False, labelbottom=False)
ax.xlabel('')

# p.ylabel('DoS')
# p.locator_params(axis='y', nbins=6)

save_plt(ax, "allmats_S_mu_temp")

In [260]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('C','mu','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=100,color=colors[i],linestyle=linestyles[i][1])

ax.xlim(eranget)
ax.ylim((-0.1e7,1.5e7))

p.legend().remove()

# p.gcf().set_size_inches(12, 3)
# plt.tick_params(axis='x', left=False, top=False, right=False, bottom=False, labelleft=False, labeltop=False, labelright=False, labelbottom=False)
# ax.xlabel('')

# p.ylabel('DoS')
p.locator_params(axis='y', nbins=4)

save_plt(ax, "allmats_C_mu_temp")

In [257]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('Po','mu','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=100,color=colors[i],linestyle=linestyles[i][1])

ax.xlim(eranget)
ax.ylim((-0.5,13))

p.legend().remove()

p.gcf().set_size_inches(12, 3)
plt.tick_params(axis='x', left=False, top=False, right=False, bottom=False, labelleft=False, labeltop=False, labelright=False, labelbottom=False)
ax.xlabel('')

# p.ylabel('DoS')
p.locator_params(axis='y', nbins=4)

save_plt(ax, "allmats_Po_mu_temp")

In [259]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('E','mu','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=100,color=colors[i],linestyle=linestyles[i][1])

ax.xlim(eranget)
ax.ylim((-1000,20000))

p.legend().remove()

p.gcf().set_size_inches(12, 3)
plt.tick_params(axis='x', left=False, top=False, right=False, bottom=False, labelleft=False, labeltop=False, labelright=False, labelbottom=False)
ax.xlabel('')

# p.ylabel('DoS')
p.locator_params(axis='y', nbins=6)

save_plt(ax, "allmats_E_mu_temp")

## Seebeck coefficient as function of...

In [77]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('S','doping','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=8,color=colors[i],linestyle=linestyles[i][1])
    
save_plt(ax, "allmats_S_doping_temp")

In [78]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('S','temp','doping',doping=dopings_arr[:1], dop_type=doping_type,ax=ax,label=k,marker=markers[i],markevery=1,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_S_temp_doping")

In [134]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('S','mu','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=100,color=colors[i],linestyle=linestyles[i][1])

ax.xlim(eranget)
ax.ylim((-300,250))

save_plt(ax, "allmats_S_mu_temp")

In [80]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('S','doping','temp',temps=temps,ax=ax)
    save_plt(ax, f"{k}_S_doping_temps")

In [81]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('S','temp','doping',doping=dopings_arr, dop_type=doping_type,ax=ax)
    save_plt(ax, f"{k}_S_temp_dopings")

In [82]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('S','mu','temp',temps=temps,ax=ax,markevery=100)
    save_plt(ax, f"{k}_S_mu_temps")

## Electronic conductivity as function of...

In [83]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('C','doping','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=8,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_C_doping_temp")

In [84]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('C','temp','doping',doping=dopings_arr[:1], dop_type=doping_type,ax=ax,label=k,marker=markers[i],markevery=1,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_C_temp_doping")

In [127]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('C','mu','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=100,color=colors[i],linestyle=linestyles[i][1])

ax.xlim(eranget)
ax.ylim((-0.1e7,1.75e7))

save_plt(ax, "allmats_C_mu_temp")

In [86]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('C','doping','temp',temps=temps,ax=ax)
    save_plt(ax, f"{k}_C_doping_temps")

In [87]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('C','temp','doping',doping=dopings_arr, dop_type=doping_type,ax=ax)
    save_plt(ax, f"{k}_C_temp_dopings")

In [88]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('C','mu','temp',temps=temps,ax=ax,markevery=100)
    save_plt(ax, f"{k}_C_mu_temps")

## Power factor as function of...

In [89]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('Po','doping','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=8,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_Po_doping_temp")

In [90]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('Po','temp','doping',doping=dopings_arr[:1], dop_type=doping_type,ax=ax,label=k,marker=markers[i],markevery=1,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_Po_temp_doping")

In [130]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('Po','mu','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=100,color=colors[i],linestyle=linestyles[i][1])

ax.xlim(eranget)
ax.ylim((-0.5,14))

save_plt(ax, "allmats_Po_mu_temp")

In [92]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('Po','doping','temp',temps=temps,ax=ax)
    save_plt(ax, f"{k}_Po_doping_temps")

In [93]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('Po','temp','doping',doping=dopings_arr, dop_type=doping_type,ax=ax)
    save_plt(ax, f"{k}_Po_temp_dopings")

In [94]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('Po','mu','temp',temps=temps,ax=ax,markevery=100)
    save_plt(ax, f"{k}_Po_mu_temps")

## Effective mass as function of...

In [96]:
ax = make_plt()
# ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('E','doping','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=8,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_E_doping_temp")

In [97]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('E','temp','doping',doping=dopings_arr[:1], dop_type=doping_type,ax=ax,label=k,marker=markers[i],markevery=1,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_E_temp_doping")

In [121]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props_mul('E','mu','temp',temps=temp,ax=ax,label=k,marker=markers[i],markevery=100,color=colors[i],linestyle=linestyles[i][1])

ax.xlim(eranget)
ax.ylim((-1000,20000))

save_plt(ax, "allmats_E_mu_temp")

In [99]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('E','doping','temp',temps=temps,ax=ax)
    save_plt(ax, f"{k}_E_doping_temps")

In [100]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('E','temp','doping',doping=dopings_arr, dop_type=doping_type,ax=ax)
    save_plt(ax, f"{k}_E_temp_dopings")

In [101]:
for k in element_data.keys():
    ax = make_plt()
    ax = element_data[k]['plotter'].plot_props('E','mu','temp',temps=temps,ax=ax,markevery=100)
    save_plt(ax, f"{k}_E_mu_temps")

# Other

In [103]:
print('\t'.join(['Temp', '\mu', 'rows', 'columns tensor']))
for p in bztTransp.Conductivity_mu, bztTransp.Seebeck_mu, bztTransp.Kappa_mu, \
         bztTransp.Effective_mass_mu, bztTransp.Power_Factor_mu, bztTransp.Carrier_conc_mu:
    print('\t'.join([str(i) for i in p.shape]))

Temp	\mu	rows	columns tensor


NameError: name 'bztTransp' is not defined

In [ ]:
bztTransp.compute_properties_doping(doping=dopings_r)

In [ ]:
print('\t'.join(['Temp', 'Doping', 'rows', 'columns tensor']))
for p in bztTransp.Conductivity_doping, bztTransp.Seebeck_doping, bztTransp.Kappa_doping, \
         bztTransp.Carriers_conc_doping,bztTransp.Effective_mass_doping, bztTransp.Power_Factor_doping:
    print('\t'.join([str(i) for i in p[doping_type].shape]))